# EDLIF model

Jaras I, Harada T, Orchard ME, Maldonado PE, Vergara RC. Extending the integrate-and-fire model to account for metabolic dependencies. Eur J Neurosci. 2021 Aug;54(4):5249-5260. doi: 10.1111/ejn.15326. Epub 2021 Jul 16. PMID: 34109698.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import nest
import numpy as np
import os

from pynestml.frontend.pynestml_frontend import generate_nest_target

NEST_SIMULATOR_INSTALL_LOCATION = nest.ll_api.sli_func("statusdict/prefix ::")


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.3
 Built: Mar 23 2022 13:33:55

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [2]:
neuron_model = ["edlif_psc.nestml", "edlif_psc_nonlineardendrite.nestml"]
neuron_id = 1
generate_nest_target(input_path=neuron_model[neuron_id],
                     target_path="/tmp/nestml-component",
                     logging_level="WARNING",
                     codegen_opts={"nest_path": NEST_SIMULATOR_INSTALL_LOCATION})

line 48:21 extraneous input '=' expecting {'integer', 'real', 'string', 'boolean', 'void', '(', NAME, UNSIGNED_INTEGER}
line 48:31 missing '=' at '('
line 48:41 mismatched input ',' expecting ')'
line 48:52 mismatched input ')' expecting '='
line 49:20 extraneous input '=' expecting {'integer', 'real', 'string', 'boolean', 'void', '(', NAME, UNSIGNED_INTEGER}
line 49:30 missing '=' at '('
line 49:44 mismatched input ',' expecting ')'
line 49:55 mismatched input ')' expecting '='


[5,GLOBAL, ERROR]: Error occurred during parsing: abort


-- The CXX compiler identification is GNU 10.3.0
-- Check for working CXX compiler: /home/wis/anaconda3/envs/wnestml/bin/c++
-- Check for working CXX compiler: /home/wis/anaconda3/envs/wnestml/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done

-------------------------------------------------------
nestmlmodule Configuration Summary
-------------------------------------------------------

C++ compiler         : /home/wis/anaconda3/envs/wnestml/bin/c++
Build static libs    : OFF
C++ compiler flags   : -fvisibility-inlines-hidden -std=c++17 -fmessage-length=0 -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/wis/anaconda3/envs/wnestml/include
NEST compiler flags  : -fvisibility-inlines-hidden -std=c++17 -fmessage-length=0 -march=nocon

In [3]:
nest.Install("nestmlmodule")


May 03 18:28:21 Install [Info]: 
    loaded module nestmlmodule


In [4]:
nest.set_verbosity("M_WARNING")
nest.ResetKernel()

neuron = nest.Create(neuron_model[neuron_id].split(".")[0])
voltmeter = nest.Create("voltmeter")

voltmeter.set({"record_from": ["new_spike", "V_m", "ATP", "ATP_c", "ATP_s"]})
nest.Connect(voltmeter, neuron)

cgs = nest.Create('dc_generator')
cgs.set({"amplitude": 1.86})
nest.Connect(cgs, neuron)

sr = nest.Create("spike_recorder")
nest.Connect(neuron, sr)

nest.Simulate(250.)

spike_times = nest.GetStatus(sr, keys='events')[0]['times']

fontsize = 15
fig, ax = plt.subplots(nrows=4, figsize=(15,15))
ax[0].plot(voltmeter.get("events")["times"], voltmeter.get("events")["V_m"])
ax[0].plot(voltmeter.get("events")["times"], voltmeter.get("events")["new_spike"])
ax[1].plot(voltmeter.get("events")["times"], voltmeter.get("events")["ATP"])
ax[2].plot(voltmeter.get("events")["times"], voltmeter.get("events")["ATP_c"])
ax[3].plot(voltmeter.get("events")["times"], voltmeter.get("events")["ATP_s"])
ax[0].scatter(spike_times, -50 * np.ones_like(spike_times), marker="d", c="orange", alpha=.8, zorder=99)
for _ax in ax:
    _ax.grid(True)
ax[0].set_ylabel("v [mV]", fontsize=fontsize)
ax[1].set_ylabel("ATP", fontsize=fontsize)
ax[2].set_ylabel("ATP_c", fontsize=fontsize)
ax[3].set_ylabel("ATP_s", fontsize=fontsize)
ax[-1].set_xlabel("Time [ms]", fontsize=fontsize)
fig.show()

NESTErrors.UnknownModelName: UnknownModelName in SLI function Create_l_i: /edlif_psc_nonlineardendrite is not a known model name.